# Import and setting

In [ ]:
# ---------- import
import gzip
import pickle

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.animation import FuncAnimation
import numpy as np

from IPython.display import HTML    # for animation in jupyter lab

In [ ]:
rcParams_dict = {
    # ---------- figure
    'figure.figsize': [8, 6],
    'figure.dpi': 120,
    'figure.facecolor': 'white',
    # ---------- axes
    'axes.grid': True,
    'axes.linewidth': 1.5,
    # ---------- ticks
    'xtick.direction': 'in',
    'ytick.direction': 'in',
    'xtick.major.width': 1.0,
    'ytick.major.width': 1.0,
    'xtick.major.size': 8.0,
    'ytick.major.size': 8.0,
    # ---------- lines
    'lines.linewidth': 2.5,
    'lines.markersize': 12,
    # ---------- grid
    'grid.linestyle': ':',
    # ---------- font
    'font.family': 'Times New Roman',
    'mathtext.fontset': 'cm',
    #'mathtext.fontset': 'stix',
    'font.size': 20,
    'axes.labelsize': 26,
    'legend.fontsize': 26,
    'svg.fonttype': 'path',  # Embed characters as paths
    #'svg.fonttype': 'none',  # Assume fonts are installed on the machine
    'pdf.fonttype': 42,  # embed fonts in PDF using type42 (True type)
}

plt.rcParams.update(rcParams_dict)

# Data

In [ ]:
def load_pkl(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def load_pkl_gz(filename):
    with gzip.open(filename, 'rb') as f:
        return pickle.load(f)

In [ ]:
rslt_data = load_pkl('./pkl_data/rslt_data.pkl')
#rslt_data = load_pkl_gz('./pkl_data/rslt_data.pkl.gz')    # for gzip

# ---------- sort Selection
#rslt_data.head(10)

# ---------- sort by Energy
rslt_data.sort_values(by=['E_eV_atom']).head(10)

In [ ]:
# ---------- Number of structures
ndata = len(rslt_data)
print(f'Number of data: {ndata}')

# ---------- check success and error
nsuccess = rslt_data['E_eV_atom'].count()
nerror = ndata - nsuccess
print(f'Success: {nsuccess}')
print(f'Error: {nerror}')

# ---------- minimum
Emin = rslt_data['E_eV_atom'].min()
print(f'Emin: {Emin} eV/atom')

In [ ]:
id_queueing, id_running, id_select_hist = load_pkl('pkl_data/LAQA_id_data.pkl')
#id_queueing, id_running, id_select_hist = load_pkl_gz('pkl_data/LAQA_id_data.pkl.gz')    # for gzip

In [ ]:
tot_step_select, laqa_step, laqa_struc, laqa_energy, laqa_bias, laqa_score = load_pkl('./pkl_data/LAQA_data.pkl')
#tot_step_select, laqa_step, laqa_struc, laqa_energy, laqa_bias, laqa_score = load_pkl_gz('./pkl_data/LAQA_data.pkl.gz')    # for gzip

# Energy vs. trial

In [ ]:
fig, ax = plt.subplots()

# ---------- axis
dx = 1
ax.set_xlim([0, ndata+dx])
ax.set_ylim([-0.2, 2])

# ---------- hline at zero
ax.hlines(0.0, -dx, ndata+dx, 'k', '--')

# ---------- plot
# x <-- ID + 1
ax.plot(rslt_data.index + 1, rslt_data['E_eV_atom'] - Emin, 'o', ms=15, mew=2.0, alpha=0.8)

# ---------- title and label
ax.set_title('LAQA for Si$_{8}$')
ax.set_xlabel('Number of structures')
ax.set_ylabel('Energy (eV/atom)')

In [ ]:
# ---------- save figure
fig.savefig('Si8_LAQA.png', bbox_inches='tight')    # PNG
#fig.savefig('title_LAQA.png', bbox_inches='tight', dpi=300)    # high dpi PNG
#fig.savefig('title_LAQA.pdf', bbox_inches='tight')    # PDF

## LAQA process

In [ ]:
# ---------- top 10
rslt_data.sort_values(by=['E_eV_atom']).head(10)

In [ ]:
# select the structure IDs you want to stand out by changing the color in the graph
stable_IDs = [12, 29, 21]    # manual input

## Required optimization steps

In [ ]:
id_done = rslt_data.index.values

In [ ]:
# only completed structures
req_step = {}
tot_req_step = 0
for cID in id_done:
        req_step[cID] = sum(laqa_step[cID])
        tot_req_step += sum(laqa_step[cID])

In [ ]:
# number of steps in each ID
req_step

In [ ]:
print('Total optimization steps:', sum(tot_step_select))
print('Total optimization steps for completed structures:', tot_req_step)
print('Number of completed structures:', len(id_done))    # include skip
print('Average number of optimization steps for completed structures:', tot_req_step/len(id_done))

## Energy vs. step for png figure

In [ ]:
fig, ax = plt.subplots()

# ---------- axis
# ------ x axis
dx = 2
sps = 10    # step per selection
ax.set_xlim([sps-dx, max(req_step.values())+dx])
major_x = MultipleLocator(20)
minor_x = MultipleLocator(10)
ax.xaxis.set_major_locator(major_x)
ax.xaxis.set_minor_locator(minor_x)
# ------ y axis
ax.set_ylim([-0.2, 2.0])

# ---------- gird
ax.grid(which='minor')    # grid: major --> minor

# ---------- title and label
ax.set_title('LAQA for Si$_{8}$')
ax.set_xlabel('Number of step')
ax.set_ylabel('Energy (eV/atom)')
#plt.tight_layout()

# ---------- hline at zero
ax.hlines(0.0, 0, max(req_step.values())+dx, 'k', '--')

# ---------- plot
# ------ not stalbe structures
for cID in range(len(laqa_energy)):
    if cID == stable_IDs:
        pass
    else:
        ax.plot(np.cumsum(laqa_step[cID]), laqa_energy[cID] - Emin, color='royalblue', linewidth=1.5)
# ------ stable structures, bring to front
for cID in stable_IDs:
    ax.plot(np.cumsum(laqa_step[cID]), laqa_energy[cID] - Emin, color='red')

In [ ]:
# ---------- save figure
fig.savefig('Si8_LAQA_step.png', bbox_inches='tight')    # PNG
#fig.savefig('title_RS.png', bbox_inches='tight', dpi=300)    # high dpi PNG
#fig.savefig('title_RS.pdf', bbox_inches='tight')    # PDF

## Energy gif anime

In [ ]:
# ---------- figure
fig2, ax2 = plt.subplots()

# --------- initialize
lines = []
xdata = []
ydata = []
num_select = {}
for cID in range(len(laqa_energy)):
    xdata.append([laqa_step[cID][0]])
    ydata.append([laqa_energy[cID][0] - Emin])
    num_select[cID] = 0
    if cID in stable_IDs:
        lines.append(ax2.plot([], [], color='red')[0])
    else:
        lines.append(ax2.plot([], [], color='royalblue', linewidth=1.5)[0])

dx = 2
sps = 10    # step per selection
ax2.set_xlim([sps-dx, max(req_step.values())+dx])
major_x = MultipleLocator(20)
minor_x = MultipleLocator(10)
ax2.xaxis.set_major_locator(major_x)
ax2.xaxis.set_minor_locator(minor_x)
ax2.grid(which='minor')    # grid: major --> minor
ax2.set_ylim([-0.2, 2.0])
ax2.set_title('LAQA for Si$_{8}$')
ax2.set_xlabel('Number of step')
ax2.set_ylabel('Energy (eV/atom)')
ax2.hlines(0.0, 0, max(req_step.values())+dx, 'k', '--')


def init():
    return lines

# ---------- animate function
# frame i --> 0, 1, 2, ...
def animate(i):
    for cID in id_select_hist[i]:
        num_select[cID] += 1
        xdata[cID].append(laqa_step[cID][num_select[cID]])
        ydata[cID].append(laqa_energy[cID][num_select[cID]] - Emin)
        lines[cID].set_data(np.cumsum(xdata[cID]), ydata[cID])
    return lines


# ---------- call the animator
anim = FuncAnimation(fig2, animate, init_func=init, frames=len(id_select_hist), blit=True)
plt.close()    # not to show the figure in jupyter, only animation

# ---------- show or save: chose only one, or you face an error
#HTML(anim.to_jshtml())    # show the animation in jupyter
anim.save('Si8_LAQA_step.gif', writer='pillow')    # default: fps=5
#anim.save('LAQA.gif', writer='pillow', fps=20)